In [ ]:
import warnings
warnings.filterwarnings("ignore")
import os
import sys
import numpy as np
import rasterio
from tqdm.notebook import tqdm
import pandas as pd
import rasterio
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as v2
sys.path.append('../prithvi/')
from utils import set_seed, load_raster
torch.multiprocessing.set_sharing_strategy('file_system')

In [ ]:
class TestDataset(Dataset):
    def __init__(self, dir_sentinel, dir_landsat, dir_bioclim, metadata, cov_columns, subset, num_classes=None, transform_sentinel=None, transform_landsat=None):
        # transform_landsat corresponds to landsat and bioclim cubes combined
        self.subset = subset
        self.transform_sentinel = transform_sentinel
        self.std_sentinel = np.array(std_sentinel)
        self.transform_landsat = transform_landsat
        self.dir_sentinel = dir_sentinel
        self.dir_landsat = dir_landsat
        self.dir_bioclim = dir_bioclim
        self.metadata = metadata
        self.cov_columns = cov_columns
        self.num_classes = num_classes
        if self.subset == "test":
            self.landsat_file_sep = "_"
        elif self.subset == "train":
            self.landsat_file_sep = "-"

    def __len__(self):
        return len(self.metadata)
    
    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        cov = torch.tensor(self.metadata.loc[idx, self.cov_columns].values.astype(np.float32))
        lonlat = torch.tensor(self.metadata.loc[idx, ["lon","lat"]].values.astype(np.float32))
        path_landsat = os.path.join(self.dir_landsat, f"GLC25-PA-{self.subset}-landsat{self.landsat_file_sep}time{self.landsat_file_sep}series_{survey_id}_cube.pt")
        sample_landsat = torch.nan_to_num(torch.load(path_landsat, weights_only=True)[:,:,:landsat_year_len])
        path_bioclim = os.path.join(self.dir_bioclim, f"GLC25-PA-{self.subset}-bioclimatic_monthly_{survey_id}_cube.pt")
        sample_bioclim = torch.nan_to_num(torch.load(path_bioclim, weights_only=True))
        tmp1 = torch.reshape(sample_bioclim, [4,-1])
        tmp2 = torch.reshape(torch.cat([tmp1[:,:1], tmp1[:,:bioclim_month_len]], axis=-1), [4,landsat_year_len,4,3])
        sample_bioclim_new = torch.permute(torch.mean(tmp2, -1), [0,2,1])[:2] 
        sample_landsat = torch.cat([sample_landsat, sample_bioclim_new], 0)
        if self.transform_landsat:
            sample_landsat = self.transform_landsat(sample_landsat)
        sample_landsat = torch.mean(sample_landsat, [-1])
        dir1, dir2 = str(survey_id)[-2:], str(survey_id)[-4:-2]        
        path_sentinel = os.path.join(self.dir_sentinel, dir1, dir2, f"{survey_id}.tiff")
        sample_sentinel = torch.nan_to_num(torch.from_numpy(load_raster(path_sentinel)).to(torch.float32))
        if self.transform_sentinel:
            sample_sentinel = self.transform_sentinel(sample_sentinel)
        sample_sentinel = torch.mean(sample_sentinel, [-1, -2])
        return sample_sentinel, sample_landsat, cov, lonlat, survey_id

class TrainDataset(TestDataset):
  def __init__(self, dir_sentinel, dir_landsat, dir_bioclim, metadata, cov_columns, label_dict, subset, num_classes, transform_sentinel=None, transform_landsat=None,
              mean_sentinel=np.array([0.0]), std_sentinel=np.array([1.0])):
    super(TrainDataset, self).__init__(dir_sentinel, dir_landsat, dir_bioclim, metadata, cov_columns, subset, num_classes, transform_sentinel, transform_landsat)
    self.label_dict = label_dict

  def __getitem__(self, idx):
    sample_sentinel, sample_landsat, cov, lonlat, survey_id = super(TrainDataset, self).__getitem__(idx)
    species_ids = self.label_dict.get(survey_id, [])  # Get list of species IDs for the survey ID
    label = torch.zeros(self.num_classes).scatter(0, torch.tensor(species_ids), torch.ones(len(species_ids)))
    return sample_sentinel, sample_landsat, cov, lonlat, label, survey_id

In [ ]:
batch_size = 32
num_workers = 4
pa_presence_threshold = 1
num_classes_total = 11255
landsat_year_len = 18
bioclim_month_len = landsat_year_len*12-1
validation_prop = 0.1
sel_countries = ["France", "Denmark", "Netherlands", "Italy"] #, "Austria"
cov_countries = 1
cov_area, cov_elevation, cov_snow = 1, 1, 1
cov_soil, cov_worldcover, cov_landcover = 1, 1, 1
path_data = "/home/gt/DATA/geolifeclef-2025"

mean_landsat = 1*np.array([ 15.0698,   16.0923,    7.9312,   68.9794,   47.9505,   24.8804, 7089.4349, 2830.6658])
std_landsat =  1*np.array([ 11.7218,   10.2417,    9.6499,   18.7112,   13.1681,    9.2436, 3332.3618,   56.7270])
mean_sentinel = 1*np.array([ 624.8547,  684.7646,  456.7674, 2924.1753])
std_sentinel =  1*np.array([ 416.0408,  351.1005,  315.8956,  943.6141])

class HorizontalCycleTransform(nn.Module):
    def forward(self, img):
        img2 = torch.cat([img, img], -1)
        start = torch.randint(img.shape[-1], (1,))[0]
        new_img = img2[:,:,start:start+img.shape[-1]]
        return new_img

transform_landsat = v2.Compose([
    v2.Normalize(mean_landsat, std_landsat),
])
transform_landsat_test = v2.Compose([
    v2.Normalize(mean_landsat, std_landsat),
])
transform_sentinel = v2.Compose([
    v2.Normalize(mean_sentinel, std_sentinel),
])
transform_sentinel_test = v2.Compose([
    v2.Normalize(mean_sentinel, std_sentinel)
])

In [ ]:
set_seed(42)
train_path_sentinel = os.path.join(path_data, "SatelitePatches/PA-train")
train_path_landsat = os.path.join(path_data, "SateliteTimeSeries-Landsat/cubes/PA-train")
train_path_bioclim = os.path.join(path_data, "BioclimTimeSeries/cubes/PA-train")
train_metadata = pd.read_csv(os.path.join(path_data, "GLC25_PA_metadata_train.csv"))
train_metadata = train_metadata.dropna(subset="speciesId").reset_index(drop=True)
train_metadata['speciesId'] = train_metadata['speciesId'].astype(int)
train_metadata["speciesIdOrig"] = train_metadata['speciesId']
tmp = train_metadata["speciesId"].value_counts() >= pa_presence_threshold
train_metadata.loc[~train_metadata["speciesId"].isin(tmp[tmp].index), "speciesId"] = -1
sp_categorical = train_metadata["speciesId"].astype("category").values
num_classes = len(sp_categorical.categories)
train_metadata['speciesId'] = sp_categorical.codes

tmp = train_metadata.groupby("surveyId").agg({"surveyId":"first", "lat":"first", "lon":"first", "areaInM2":lambda x: list(x.unique()), "region":"first", "country":"first", "speciesId":list})
train_label_series = tmp.set_index("surveyId").speciesId
train_metadata = tmp.drop(columns=["speciesId"]).set_index("surveyId", drop=False)
train_metadata["area"] = train_metadata["areaInM2"].apply(lambda x: 1.0 if np.isinf(x).all() else np.mean(x, where=~np.isinf(x)))
train_metadata["areaLog"] = np.log10(train_metadata["area"])

train_metadata['area'].fillna(train_metadata['area'].mean(), inplace=True)
train_metadata['areaLog'].fillna(train_metadata['areaLog'].mean(), inplace=True)
country_columns = ["con"+country[:3] for country in sel_countries] + ["conOther"]
for country, col in zip(sel_countries, country_columns[:-1]):
    train_metadata[col] = train_metadata["country"] == country
train_metadata[country_columns[-1]] = ~train_metadata["country"].isin(sel_countries)
train_elevation = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "Elevation", "GLC25-PA-train-elevation.csv"), index_col=0)
train_elevation['Elevation'].fillna((train_elevation['Elevation'].mean()), inplace=True)
train_soil = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "SoilGrids", "GLC25-PA-train-soilgrids.csv"), index_col=0)
for column in train_soil.columns: train_soil[column].fillna((train_soil[column].mean()), inplace=True)
train_worldcover = pd.read_csv(os.path.join(path_data, "worldcover", "s2_pa_train_survey_points_with_worldcover.csv"), index_col=0)
train_wcdummy = pd.get_dummies(train_worldcover["class"], prefix="wc")
train_wcdummy.drop(columns="wc_70", inplace=True)
train_wcdummy.drop(columns="wc_100", inplace=True)
train_landcover = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "LandCover", "GLC25-PA-train-landcover.csv"), index_col=0)
landcover_col_ind = [0,2,3,5,8,11,12]
train_landcover = train_landcover.iloc[:, landcover_col_ind]
train_snow = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "chelsa_snow", "pa_train_snowcover_chelsa_scd.csv"), index_col=0).sort_index()

cov_flag_list = [cov_area, cov_elevation, cov_countries, cov_soil, cov_worldcover, cov_landcover, cov_snow]
cov_name_list = [["areaLog"], ["Elevation"], country_columns, list(train_soil.columns), list(train_wcdummy.columns), list(train_landcover.columns), list(train_snow.columns)]
cov_columns = sum([flag*name for flag, name in zip(cov_flag_list, cov_name_list)], [])
print(cov_columns)
train_df_list = [train_metadata, train_elevation, train_soil, train_wcdummy, train_landcover, train_snow]
print("All rows match: ", [(train_df_list[0].index==df.index).all() for df in train_df_list[1:]])
train_combined = pd.concat(train_df_list, axis=1)
cov_norm_coef = train_combined.loc[:,cov_columns].agg(['mean', 'std'])
dummy_columns = country_columns + list(train_wcdummy.columns)
cov_norm_coef.loc["mean",dummy_columns] = 0
cov_norm_coef.loc["std",dummy_columns] = 1
train_combined.loc[:,cov_columns] = (train_combined.loc[:,cov_columns] - cov_norm_coef.loc["mean"]) / cov_norm_coef.loc["std"]

In [ ]:
#cov_norm_coef.to_csv(os.path.join(path_data, "hmsc", "train_cov_mean_std.csv"))

In [ ]:
train_data = train_combined.reset_index(drop=True)
train_label_dict = train_label_series.to_dict()
train_dataset = TrainDataset(train_path_sentinel, train_path_landsat, train_path_bioclim, train_data, cov_columns, train_label_dict, 
                             subset="train", num_classes=num_classes, transform_sentinel=transform_sentinel, transform_landsat=transform_landsat)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
print(train_dataset[0][0].shape, train_dataset[0][1].shape, train_dataset[0][2].shape, train_dataset[0][3].shape, train_dataset[0][4].shape)

In [ ]:
surv_list, cov_list, y_list = [], [], []
for sentinel, landsat, cov, lonlat, label, survey in tqdm(train_loader): #range(len(train_dataset))
    #vals = train_dataset[i]
    cov = torch.concat([sentinel, landsat.reshape([landsat.shape[0], -1]), cov, lonlat], dim=1).numpy()
    cov_list.append(cov)
    y_list.append(label.numpy())
    surv_list.append(survey.numpy())

In [ ]:
cols = [f"sentinel{i}" for i in range(sentinel.shape[-1])] + [f"landsatbio{i}{j}" for i in range(landsat.shape[-2]) for j in range(landsat.shape[-1])] + cov_columns + ["lon","lat"]
cov = pd.DataFrame(np.concatenate(cov_list), columns=cols)
Y = pd.DataFrame(np.concatenate(y_list)).astype(int)
os.makedirs(os.path.join(path_data, "hmsc", ""), exist_ok=True)
cov.to_csv(os.path.join(path_data, "hmsc", "train_cov.csv"), index_label=False)
Y.to_csv(os.path.join(path_data, "hmsc", "train_Y.csv"), index_label=False)
surv = pd.DataFrame({"surveyId":np.concatenate(surv_list)})

In [ ]:
test_path_sentinel = os.path.join(path_data, "SatelitePatches/PA-test")
test_path_landsat = os.path.join(path_data, "SateliteTimeSeries-Landsat/cubes/PA-test")
test_path_bioclim = os.path.join(path_data, "BioclimTimeSeries/cubes/PA-test")
test_metadata = pd.read_csv(os.path.join(path_data, "GLC25_PA_metadata_test.csv")).set_index("surveyId", drop=False).sort_index()
test_metadata.rename(columns={"areaInM2": "area"}, inplace=True)
test_metadata.replace({"area": [np.inf, -np.inf]}, 1.0, inplace=True)
test_metadata['areaLog'] = np.log10(test_metadata['area'])
test_metadata['area'].fillna(test_metadata['area'].mean(), inplace=True)
test_metadata['areaLog'].fillna(test_metadata['areaLog'].mean(), inplace=True)
for country, col in zip(sel_countries, country_columns[:-1]):
    test_metadata[col] = test_metadata["country"] == country
test_metadata[country_columns[-1]] = ~test_metadata["country"].isin(sel_countries)
test_elevation = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "Elevation", "GLC25-PA-test-elevation.csv"), index_col=0).sort_index()
test_elevation = test_elevation.loc[test_elevation.index.isin(test_metadata.index)]
test_elevation['Elevation'].fillna((test_elevation['Elevation'].mean()), inplace=True)
test_soil = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "SoilGrids", "GLC25-PA-test-soilgrids.csv"), index_col=0).sort_index()
test_soil = test_soil.loc[test_soil.index.isin(test_metadata.index)]
for column in test_soil.columns: test_soil[column].fillna((test_soil[column].mean()), inplace=True)
test_worldcover = pd.read_csv(os.path.join(path_data, "worldcover", "pa_test_survey_points_with_worldcover.csv"), index_col=0).sort_index()
test_wcdummy = pd.get_dummies(test_worldcover["class"], prefix="wc")
test_wcdummy.drop(columns="wc_100", inplace=True)
# test_wcdummy.insert(6, "wc_70", False)
test_landcover = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "LandCover", "GLC25-PA-test-landcover.csv"), index_col=0).sort_index()
test_landcover = test_landcover.loc[test_landcover.index.isin(test_metadata.index)]
test_landcover = test_landcover.iloc[:, landcover_col_ind]
test_snow = pd.read_csv(os.path.join(path_data, "EnvironmentalValues", "chelsa_snow", "pa_test_snowcover_chelsa_scd.csv"), index_col=0).sort_index()
test_snow = test_snow.loc[test_snow.index.isin(test_metadata.index)]

test_df_list = [test_metadata, test_elevation, test_soil, test_wcdummy, test_landcover, test_snow]
print("All rows match: ", [(test_df_list[0].index==df.index).all() for df in test_df_list[1:]])
test_combined = pd.concat(test_df_list, axis=1)
test_combined.loc[:,cov_columns] = (test_combined.loc[:,cov_columns] - cov_norm_coef.loc["mean"]) / cov_norm_coef.loc["std"]
test_combined.reset_index(drop=True, inplace=True)
test_dataset = TestDataset(test_path_sentinel, test_path_landsat, test_path_bioclim, test_combined, cov_columns, subset="test", 
                           transform_sentinel=transform_sentinel_test, transform_landsat=transform_landsat_test)
print(test_dataset[0][0].shape, test_dataset[0][1].shape, test_dataset[0][2].shape, test_dataset[0][3].shape)

In [ ]:
cov_list = []
for i in tqdm(range(len(test_dataset))):
    vals = test_dataset[i]
    cov = torch.concat([vals[0], vals[1].flatten(), vals[2], vals[3]]).numpy()
    cov_list.append(cov)

In [ ]:
cols = [f"sentinel{i}" for i in range(len(vals[0]))] + [f"landsatbio{i}{j}" for i in range(vals[1].shape[0]) for j in range(vals[1].shape[1])] + cov_columns + ["lon","lat"]
cov = pd.DataFrame(cov_list, columns=cols)
os.makedirs(os.path.join(path_data, "hmsc"), exist_ok=True)
cov.to_csv(os.path.join(path_data, "hmsc", "test_cov.csv"), index_label=False)

In [ ]:
train_combined.head()